In [21]:
# parsing us patents
# patent should be downloaded from: https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/2021/
# This code is Inspired by https://github.com/lettergram/parse-uspto-xml

import pprint
import os
import sys
import html
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import csv
import re

# ---------------------------------------------------------------------------
# Function 
# ---------------------------------------------------------------------------
def parse_patent(bs):
    """
    Parses a US patent in a BeautifulSoup object.
    Only keeping the elements useful for our next text classification task
    """
    
    publication_title = bs.find('invention-title').text
    application_type = bs.find('application-reference')['appl-type']

    # International Patent Classification (IPC) Docs:
    sections = {}
    for classes in bs.find_all('classifications-ipcr'):
        for el in classes.find_all('classification-ipcr'):
            section = el.find('section').text                            
            sections[section] = True
    #print(list(sections.keys()))
    section_list = list(sections.keys())
    section_string =",".join(section_list)
        
    regex = re.compile('[^a-zA-Z]')  # remove all but alphabetic (even numbers -- change to '[^a-zA-Z0-9]' to keep numbers)
    
        
    abstracts = ''
    for el in bs.find_all('abstract'):
        abstracts += el.text.strip('\n')
        abstracts = regex.sub(' ', abstracts)
        abstracts = re.sub(r'\b\w{1,2}\b', '', abstracts)  # remove words of length < 3

    
    descriptions = ''
    for el in bs.find_all('description'):
        descriptions += el.text.strip('\n')
        descriptions = regex.sub(' ', descriptions)
        descriptions = re.sub(r'\b\w{1,2}\b', '', descriptions)  # remove words of length < 3

    
    claims = ''
    for el in bs.find_all('claim'):
        claims += el.text.strip('\n')
        claims = regex.sub(' ', claims)
        claims = re.sub(r'\b\w{1,2}\b', '', claims)  # remove words of length < 3
        #print(claims)
    
    # output in the dictionary format
    uspto_patent = {
        "sections": [section_string], 
        "publication_title": regex.sub(' ', publication_title.replace("\n"," ")),
        "application_type": regex.sub(' ', application_type.replace("\n"," ")),
        "abstract": abstracts.replace("\n"," "), 
        "descriptions": descriptions.replace("\n"," "), 
        "claims": claims.replace("\n"," ")
    }        

    return uspto_patent
# ---------------------------------------------------------------------------




# input file:
#filenames = 'tmp/ipg210105_3.xml'
filenames = 'ipg210105.xml'

print('Start ...')    
all_patents = pd.DataFrame()
n_file = 1
count = 1
success_count = 0
errors = []

print("Filename:", filenames)
print("\n--------------------------------------------------------\n")

xml_text = html.unescape(open(filenames, 'r').read())

for patent in xml_text.split("<?xml version=\"1.0\" encoding=\"UTF-8\"?>"):
    
    if patent is None or patent == "":
        print('No Patent')
        continue

    bs = BeautifulSoup(patent)

    # Skip documents which do not have classification-ipcr
    if bs.find('classification-ipcr') is None:
        #print('No classification!')
        continue 

    application = bs.find('us-patent-application')
    if application is None: # If no application, search for grant
        application = bs.find('us-patent-grant')

        
    # Setting the title
    title = "None"
    try:
        title = application.find('invention-title').text
    except Exception as e:          
        print("Error", count, e)

    # parsing the patent
    try: 
        uspto_patent = parse_patent(application)       
        new_patent = pd.DataFrame.from_dict(data=uspto_patent)
        
        if all_patents.empty:
            all_patents = new_patent            
        else:
            frames = [all_patents, new_patent]
            all_patents = pd.concat(frames)                
        success_count += 1
    except Exception as e:
        exception_tuple = (count, title, e)
        errors.append(exception_tuple)

    if (success_count+len(errors)) % 50 == 0:
        print(count, filenames, title)
        #new_patent.describe()
    count += 1

#print("\n\nErrors\n------------------------\n")
#for e in errors:
#    print(e)
    
print("Success Count:", success_count)
print("Error Count:", len(errors))

#all_patents.to_csv('all_patents_ipg210105.tsv', index=False, sep="\t")
all_patents.to_csv('all_patents_ipg210105.csv', index=False)
#all_patents.to_csv('tmp/all_patents_ipg210105_3.csv', index=False)

print('Saved to file!')


Start ...
Filename: ipg210105.xml

--------------------------------------------------------

No Patent
50 ipg210105.xml Methods for hybrid corn seed production
100 ipg210105.xml Antimicrobial compositions containing alkylpyrazines and their uses
150 ipg210105.xml Temperature-reading face shield
200 ipg210105.xml Item of furniture and wall element for an item of furniture
250 ipg210105.xml Cleaner and method for controlling cleaner
300 ipg210105.xml Devices and systems for correcting errors in blood pressure measurements
350 ipg210105.xml Systems and methods for monitoring the health of vehicle passengers using camera images
400 ipg210105.xml Positioning tool for anastomosis
450 ipg210105.xml Peri-vascular tissue ablation catheters
500 ipg210105.xml Lifetime regenerative heart valve
550 ipg210105.xml Evaluation method of wearing state of disposable diaper, evaluation system of wearing state of disposable diaper, and non-transitory computer-readable storage medium with a program for eval